In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Carica il dataset
df = pd.read_csv('../dataset/preprocessed_dataset.csv')
df = df.drop('votes', axis=1)
df = df.drop('avg_vote', axis=1)

In [64]:
X = df.drop('revenue_with_CPI', axis=1)
y = df['revenue_with_CPI']
print(X.head())

   duration  converted_budget  dir_oscar_nomination  writer_oscar_nomination  \
0      88.0          175700.3                     0                        0   
1      59.0         3013850.0                     0                        0   
2      77.0          521727.6                     0                        0   
3      50.0         5598468.6                     0                        0   
4     300.0        10802441.1                     0                        0   

   cast_globe_nomination  BAFTA_act_nom  BAFTA_dir_nom  BAFTA_writer_nom  \
0                      0              0              0                 0   
1                      0              0              0                 0   
2                      0              0              0                 0   
3                      0              0              0                 0   
4                      0              0              0                 0   

   dir_emmy_nom  writer_emmy_nom  ...  month_3  month_4  month

In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [66]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy import stats
scaler = StandardScaler()

numerical_columns = ['duration','converted_budget',
                     'dir_oscar_nomination', 'writer_oscar_nomination',
                     'cast_globe_nomination',
                     'BAFTA_writer_nom', 'BAFTA_dir_nom', 'BAFTA_act_nom', 
                     'dir_emmy_nom', 'writer_emmy_nom', 'act_emmy_nom',
                     #'actors_films_before', 'director_films_before', 'writers_films_before'
                     ]

for col in numerical_columns:
     X_train[col] = np.log1p(X_train[col])
     X_test[col] = np.log1p(X_test[col])


scaler = StandardScaler()

# Adatta lo scaler solo al training set e poi trasforma sia il training set che il test set
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])


#X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)


# Esegui la PCA solo sul set di training
#pca = PCA(n_components=10)  # Sostituisci 10 con il numero desiderato di componenti principali
#X_train_resampled = pca.fit_transform(X_train_resampled)
#X_test = pca.transform(X_test)  # Applica le stesse trasformazioni al set di test

#y_train_transformed, lambda_train = stats.boxcox(y_train)
#y_test_transformed = stats.boxcox(y_test, lmbda=lambda_train)

#y_train_transformed = np.log1p(y_train)
#y_test_transformed = np.log1p(y_test)


In [67]:
from sklearn.ensemble import RandomForestRegressor

# Addestra un modello di Random Forest
rf = RandomForestRegressor(n_estimators=100,max_features=4, random_state=42)
rf.fit(X_train, y_train)

# Ottieni le importanze delle feature
importances = rf.feature_importances_

# Seleziona le feature con importanza sopra una soglia
threshold = np.percentile(importances, 15)  # Seleziona le feature nel top 25%
selected_features_rf = X_train.columns[importances >= threshold].tolist()

X_train = X_train[selected_features_rf]
X_test = X_test[selected_features_rf]
print("Selected features based on Random Forest importance:", selected_features_rf)


Selected features based on Random Forest importance: ['duration', 'converted_budget', 'dir_oscar_nomination', 'writer_oscar_nomination', 'cast_globe_nomination', 'BAFTA_act_nom', 'BAFTA_dir_nom', 'BAFTA_writer_nom', 'dir_emmy_nom', 'writer_emmy_nom', 'act_emmy_nom', 'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Biography', 'genre_Comedy', 'genre_Crime', 'genre_Drama', 'genre_Family', 'genre_Fantasy', 'genre_Film-Noir', 'genre_History', 'genre_Horror', 'genre_Music', 'genre_Musical', 'genre_Mystery', 'genre_Romance', 'genre_Sci-Fi', 'genre_Sport', 'genre_Thriller', 'genre_War', 'genre_Western', 'language_Cantonese', 'language_Dutch', 'language_English', 'language_Finnish', 'language_French', 'language_German', 'language_Hindi', 'language_Italian', 'language_Japanese', 'language_Korean', 'language_Mandarin', 'language_Portuguese', 'language_Russian', 'language_Spanish', 'language_Turkish', 'production_company_CJ Entertainment', 'production_company_Dimension Films', 'produc

# Random Forest

In [68]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Addestra il modello Random Forest
rf_regressor = RandomForestRegressor(n_jobs=10, n_estimators=500, max_features=4)
rf_regressor.fit(X_train, y_train)

# Fai previsioni sui dati di test
y_pred_test = rf_regressor.predict(X_test)

# Fai previsioni sui dati di train
y_pred_train = rf_regressor.predict(X_train)

# Calcola le metriche di valutazione per il test set
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

# Calcola le metriche di valutazione per il train set
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)

print(f'Test Set - MSE: {mse_test}')
print(f'Test Set - R²: {r2_test}')
print(f'Test Set - MAE: {mae_test}')

print(f'\nTrain Set - MSE: {mse_train}')
print(f'Train Set - R²: {r2_train}')
print(f'Train Set - MAE: {mae_train}')


Test Set - MSE: 4.880401512064917e+16
Test Set - R²: 0.34405589669769
Test Set - MAE: 88354757.14872123

Train Set - MSE: 7656488104101678.0
Train Set - R²: 0.9092851907121938
Train Set - MAE: 31787030.893196538


# GradientBoost

In [69]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge

gb_regressor = GradientBoostingRegressor(n_estimators=500, max_depth=4, learning_rate=0.1)
gb_regressor.fit(X_train, y_train)

# Fai previsioni sui dati di test
y_pred_test = gb_regressor.predict(X_test)

# Fai previsioni sui dati di train
y_pred_train = gb_regressor.predict(X_train)

# Calcola le metriche di valutazione per il test set
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

# Calcola le metriche di valutazione per il train set
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)

print(f'Test Set - MSE: {mse_test}')
print(f'Test Set - R²: {r2_test}')
print(f'Test Set - MAE: {mae_test}')

print(f'\nTrain Set - MSE: {mse_train}')
print(f'Train Set - R²: {r2_train}')
print(f'Train Set - MAE: {mae_train}')


Test Set - MSE: 5.0923859867473016e+16
Test Set - R²: 0.3155643953702274
Test Set - MAE: 84294180.4436582

Train Set - MSE: 1.3290818490481512e+16
Train Set - R²: 0.8425291010382445
Train Set - MAE: 56600260.030522086
